 “FAO GAUL” FeatureCollection과 “Sentinel-2” ImageCollection을 불러오고, 전라북도의 2025년 계절별 NDVI 이미지 export 하기(결과물 총 4장의 tif 파일)

# Get started with Earth Engine for Python

This quickstart will give you an interactive introduction to visualizing and
analyzing geospatial data with the Earth Engine Python interface.

## Before you begin

[Register or create](https://code.earthengine.google.com/register) a Google Cloud Project; you'll be prompted to complete the following steps. If you already have a project registered for Earth Engine access, skip to the next section.

  * Select the project's purpose: commercial or noncommercial.
  * If the purpose is noncommercial, select a project type.
  * Create a new Google Cloud project or select an existing project.
  * If the purpose is commercial, verify or set up billing for your project.
  * Confirm your project information.  

**Note:** If you don't plan to keep the resources that you create in this procedure, create a project instead of selecting an existing project. After you finish these steps, you can [delete the project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects), removing all resources owned by the project.

## Notebook setup

**1.** Import the Earth Engine and geemap libraries.

In [ ]:
import ee
import geemap.core as geemap
import geemap


**2.** Authenticate and initialize the Earth Engine service. Follow the
resulting prompts to complete authentication. Be sure to replace PROJECT_ID
with the name of the project you set up for this quickstart.

In [ ]:
ee.Authenticate()
ee.Initialize(project='ee-gmldnjs295')

## FAO GAUL 행정경계 불러오기 (전라북도 추출)

In [ ]:
FAO_GAUL = ee.FeatureCollection('FAO/GAUL/2015/level1')


FAO_jeonbuk = (
    FAO_GAUL
    .filter(ee.Filter.eq('ADM0_NAME', 'Republic of Korea'))
    .filter(ee.Filter.eq('ADM1_NAME', 'Chollabuk-do'))
)



Map = geemap.Map(center=[36.5, 127.8], zoom=6)
styleParams = {
    'fillColor': 'b5ffb4',
    'color': '00909F',
    'width': 1.0
}
styled_dataset = FAO_jeonbuk.style(**styleParams)

Map.addLayer(styled_dataset, {}, 'First Level Administrative Units')
Map


Map(center=[36.5, 127.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

## Sentinel-2 ImageCollection 불러오기

In [ ]:
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

def mask_clouds(image):
    qa = image.select('QA60')

    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    mask = (
        qa.bitwiseAnd(cloud_bit_mask).eq(0)
        .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    )

    return (
        image
        .updateMask(mask)
        .divide(10000)  # reflectance scaling
        .copyProperties(image, ['system:time_start'])
    )


In [ ]:
def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

seasons = {
    'spring': ('2025-03-01', '2025-05-31'),
    'summer': ('2025-06-01', '2025-08-31'),
    'autumn': ('2025-09-01', '2025-11-30'),
    'winter': ('2024-12-01', '2025-02-28')  # 겨울 주의
}

def seasonal_ndvi(start_date, end_date):
    return (
        s2
        .filterDate(start_date, end_date)
        .filterBounds(FAO_jeonbuk)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
        .map(mask_clouds)
        .map(add_ndvi)
        .select('NDVI')
        .median()
        .clip(FAO_jeonbuk)
    )

ndvi_spring = seasonal_ndvi(*seasons['spring'])
ndvi_summer = seasonal_ndvi(*seasons['summer'])
ndvi_autumn = seasonal_ndvi(*seasons['autumn'])
ndvi_winter = seasonal_ndvi(*seasons['winter'])

ndvi_spring


In [25]:
ndvi_vis = {
    'min': 0,
    'max': 1,
    'palette': ['white', 'yellow', 'green']
}

import geemap

Map = geemap.Map()
Map.centerObject(FAO_jeonbuk, 8)

Map.addLayer(ndvi_spring, ndvi_vis, 'Spring NDVI')
Map.addLayer(ndvi_summer, ndvi_vis, 'Summer NDVI')
Map.addLayer(ndvi_autumn, ndvi_vis, 'Autumn NDVI')
Map.addLayer(ndvi_winter, ndvi_vis, 'Winter NDVI')

Map

Map(center=[35.69489055462313, 127.16022176311671], controls=(WidgetControl(options=['position', 'transparent_…

In [26]:
task = ee.batch.Export.video.toDrive(
    collection=ndvi_vis,
    description='NDVI_Jeonbuk_2025_Animation',
    folder='GEE_Animation',
    fileNamePrefix='NDVI_Jeonbuk_2025',
    region=FAO_jeonbuk.geometry(),
    scale=10,
    framesPerSecond=2,
    crs='EPSG:4326'
)
task.start()

AttributeError: 'dict' object has no attribute 'prepare_for_export'

In [ ]:
def export_png(image, name):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=name,
        folder='GEE_NDVI_Jeonbuk_2025_PNG',
        fileNamePrefix=name,
        region=FAO_jeonbuk.geometry(),
        scale=10,
        crs='EPSG:4326',
        maxPixels=1e13,
        fileFormat='PNG'
    )
    task.start()

ndvi_png = ndvi_spring.visualize(**ndvi_vis)
export_png(ndvi_png, 'Jeonbuk_2025_Spring_NDVI_PNG')


EEException: ImageFileFormat "PNG" is invalid for this export type. Valid ImageFileFormats: ["GEO_TIFF", "TF_RECORD_IMAGE"].

In [ ]:
season_images = {
    'Spring': ndvi_spring,
    'Summer': ndvi_summer,
    'Fall': ndvi_autumn,
    'Winter': ndvi_winter
}

for season, img in season_images.items():
    task = ee.batch.Export.image.toDrive(
        image=img,
        description=f'NDVI_Jeonbuk_{season}_2025',
        folder='GEE_NDVI_2025',
        fileNamePrefix=f'NDVI_Jeonbuk_{season}_2025',
        region=FAO_jeonbuk.geometry(),
        scale=10,
        crs='EPSG:4326',
        fileFormat='GeoTIFF',
        maxPixels=1e13
    )
    task.start()
    print(season)


Spring
Summer
Fall
Winter


In [ ]:
ndvi_vis = ndvi_spring.visualize(
    min=0,
    max=1,
    palette=['white', 'yellow', 'green']
)
boundary_img = ee.Image().paint(
    featureCollection=FAO_jeonbuk,
    color=1,
    width=2
)
boundary_vis = boundary_img.visualize(
    palette=['red']
)
composite = ndvi_vis.blend(boundary_vis)
url = composite.getThumbURL({
    'region': FAO_jeonbuk.geometry(),
    'dimensions': 1024,
    'format': 'png'
})

print(url)


https://earthengine.googleapis.com/v1/projects/ee-gmldnjs295/thumbnails/0ef55d9de97161c27b51af127149106e-2c06b7e6c5b9658d99f9b71783d971b9:getPixels


In [ ]:
task = ee.batch.Export.image.toDrive(
    image=ndvi_spring,                     # 저장할 이미지 (ee.Image)
    description='NDVI_Jeonbuk_Spring_2025',# 작업 이름
    folder='GEE_NDVI_2025',                # Drive 폴더명
    fileNamePrefix='NDVI_Jeonbuk_Spring',  # 파일명 (확장자 제외)
    region=FAO_jeonbuk.geometry(),         # 저장 영역
    scale=10,                              # 해상도 (Sentinel-2: 10m)
    crs='EPSG:4326',                       # 좌표계 (WGS84)
    fileFormat='GeoTIFF',                  # ★ tif 형식
    maxPixels=1e13
)

task.start()
